<a href="https://colab.research.google.com/github/bansi2/Machine_Learing_Projects/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fake News Prediction **

In this project we will be predicting either a news is fake or real. In this project we will be using the LOGISTICS REGRESSION MODEL to get the prediction


About The Dataset:
1.    Id : unique id for a news articale
2.   title : the title of a news article
3.   Lauthor: author of the news article
4.   text: the text of the article;could be incomplete
5.   List itemlabel: a label that makes wheather the news article is real or fake





# Importing all the required depndencies

In [4]:
import numpy as np
import pandas as pd
import re  #regular expression used for searching words
from nltk.corpus import stopwords # used to find the words which doesn't add much value to article {eg: where, was, etc}
from nltk.stem.porter import PorterStemmer  #used to remove suffix and prefix
from sklearn.feature_extraction.text  import TfidfVectorizer # used to convert text into feature vectors(numbers)
from sklearn.model_selection import train_test_split #used to train and test the data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score #for scoring model

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english')) # this are the stopwords which will be removed from the article

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

#Data Pre-Processing

In [9]:
#loading the dataset in dataframe
news_dataset = pd.read_csv('/content/train.csv')

In [10]:
news_dataset.shape

(20800, 5)

In [11]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()


,0
id,0
title,558
author,1957
text,39
label,0


In [13]:
#As we have the large dataset we can replace the null values with the empty string
news_dataset = news_dataset.fillna(' ')  #fillna will fill the missing values

In [14]:
news_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [38]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']


In [39]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [46]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']


In [47]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [42]:
#Stemming : The process of reducing the word to its root word
#eg : actor, actress, acting ----> act

In [48]:
port_stem = PorterStemmer()

In [49]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [50]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [51]:
print(news_dataset['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [52]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values


In [54]:
print(X)


['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [56]:
print(Y) #1 = Fake News and  0 = Real news

[1 0 1 ... 0 1 1]


In [57]:
Y.shape

(20800,)

In [60]:
#As the Values of X are in character so we will vectorize it to convert it in numerical
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [61]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210584 stored elements and shape (20800, 16984)>
  Coords	Values
  (0, 263)	0.2701012497770876
  (0, 2464)	0.36765196867972083
  (0, 2933)	0.24684501285337127
  (0, 3567)	0.3598939188262558
  (0, 3759)	0.27053324808454915
  (0, 4917)	0.23331696690935097
  (0, 6930)	0.2187416908935914
  (0, 7612)	0.24785219520671598
  (0, 8546)	0.2921251408704368
  (0, 8822)	0.36359638063260746
  (0, 13348)	0.2565896679337956
  (0, 15553)	0.2848506356272864
  (1, 1481)	0.2957471154505952
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 2790)	0.19208753385709676
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 16656)	0.3025156488372128
  (2, 2917)	0.3179886800654691
  (2, 3072)	0.46097489583229645
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  :	:
  (20797, 3610)	0.211655450844435
  (20797, 6969)	0.21809398920480086
  (20797

Splitting the data into train and test dataset




1.   test_size=0.2: This argument specifies that 20% of the data should be allocated to the test set. The remaining 80% will be used for training.
2.   stratify=Y: This is important for maintaining the proportion of classes (real and fake news) in both the training and testing sets. It ensures that the split reflects the original distribution of your target variable. For example, if your original dataset has 60% real news and 40% fake news, both your training and testing sets will have roughly the same proportion.
3.  random_state=2: This ensures that the split is reproducible. If you run the code multiple times with the same random_state value, you'll get the same split each time. This is helpful for consistency in your experiments.






In [62]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state= 2)

Training our Logistic regression model

In [64]:
model = LogisticRegression()

In [65]:
#now we need to train the model
model.fit(X_train,Y_train)

LogisticRegression()

# Evaluation of Model

 Accuracy score of the training data

In [74]:
#Accuracy Score on the training data
X_train_prediction = model.predict(X_train) #model.predict will predict the result for X_train and save the values in X_train_prediction
training_data_accuracy = accuracy_score(X_train_prediction, Y_train) #accuracy_score will compare the results of X_train_prediction and Y_train and give the accuracy score


In [72]:
print(training_data_accuracy)

0.9863581730769231


 Accuracy score of test data

In [75]:
#Accuracy Score on the test data
X_test_prediction = model.predict(X_test) #model.predict will predict the result for X_test and save the values in X_test_prediction
test_data_accuracy = accuracy_score(X_test_prediction, Y_test) #accuracy_score will compare the results of X_test_prediction and Y_test and give the accuracy score

In [76]:
print(test_data_accuracy)

0.9790865384615385


# Making a Predictive System

In [87]:
print(Y_test[3999])

1


In [89]:
X_new = X_test[3999]
prediction = model.predict(X_new)

if(prediction[0] == 0):
  print("The news is Real")
else:
  print("The news is Fake")

The news is Fake
